![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)
 
<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Flesson-plans&branch=anywidgets&urlpath=notebooks/lesson-plans/notebooks/plants/PhidgetFour_auto.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"></a>

# Watering plants with Phidgets (Automatically)

This is a short notebook to collect and display data from "Phidgets" sensors around a plant and allow the user to automatically water the plant on a daily basis.

<h2 style="color: red">WARNING</h2>
Water can cause all kinds of damage. An automatic watering system can lead to accidental spills that could damage your computer, your furniture and more. Be sure to set up your plant watering system someplace safe, where a water spill will not cause trouble.

One safe way is to place the plant and watering system in a waterproof plastic container. 

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called [Phidgets](https://www.phidgets.com/education/learn/projects/plant-kit/). This kit has all the sensors we need to monitor the environment of the plant, along with a water pump and control. The data can be viewed directly on the computer screen, and the water pump can be activated to water the plant.

- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- you will need a computer (Mac, Windows, Raspberry Pi) with a USB connection and internet access
- you will need the Chrome or Chromium Web browser (Safari, Firefox, and Edge will not work)

Follow these steps:

Assemble your Phidgets plant kit, see the following [link](https://www.phidgets.com/education/learn/projects/plant-kit/assemble/)

1. Setting up the interface
1. View the interface
1. Connecting the sensor devices
1. Watering your plant
1. Observing the results 


Run the cells in this notebook, one cell at a time. This will give you the chance to respond to any errors and fix things if necessary. Do not select "Run All." 

You should have a plant, with the sensors, water reservoir, and pump  nearby. This what the plant setup should look like:

<div align="center">
<img src="images/plant-water.jpg" alt="A plant with sensors and water pump" width="400"/><br>
A plant with sensors and water pump.
</div>

Here is some more detail on the various devices we are using. 
<div align="center">
<img src="images/plant-water.jpg" alt="A plant with sensors and water pump" width="400"/><br>
1: Pump. 2: DC power supply. 3: Moisture probe. 4: Light sensor. 5: Humidity sensor. 6: VINT Hub
</div>

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it. 

***
## Step 1. Setting up the user interface

Our goal is to create a user interface that looks like this:


<div align="center">
<img src="images/AutoWaterUI.jpg" alt="Image of the user interface" width="800"/><br>
A plant with sensors and water pump.
</div>

This UI includes a button to connect the Phidget, four gauges to display values for temperature, humidity, soil moisture and light levels, and some buttons to water the plant. 

We have two watering buttons, one immediately water the plant, the other to to start/stop the automatic watering. 

There is also a slider, to adjust the moisture setpoint. The plant will only be watered if the moisture sensor goes below this level. 

Finally, there is a text box to record when the plant was watered. 

## Step 1a. 

Let's set up the software widget called "PhidgetFive_auto" that will to connect the Phidget hardware.

In [ ]:
# Special libraries for the anywidgets
import pathlib
import traitlets
try:
    import anywidget
except:
    !pip install --user "anywidget[dev]"
    import anywidget

In [ ]:
## define the class
class PhidgetFive_auto(anywidget.AnyWidget):
    _esm = pathlib.Path("ph5auto.js")
    _css = pathlib.Path("ph4xx.css")
    # These 5 variables are used by the widget class. Do not delete. 
    hubPort = traitlets.Int(1).tag(sync=True)   # default is port 1 on Phidget Plant Kit
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)
    pump = traitlets.Bool(False).tag(sync=True)

## create the PhidgetFour widget and name it pf
pf = PhidgetFive_auto()
pf.hubPort = 1   ## The water pump is plugged into Port 1 (you can change this if you use another port)

# connect the Phidget widget to the gauges in the user interface
def updateTemperature(change):
    gauges.data[0]['value'] = change.new
def updateHumidity(change):
    gauges.data[1]['value'] = change.new
def updateMoisture(change):
    gauges.data[2]['value'] = change.new
def updateLuminance(change):
    gauges.data[3]['value'] = change.new

pf.observe(updateTemperature, names=["temperature"])
pf.observe(updateHumidity, names=["humidity"])
pf.observe(updateMoisture, names=["moisture"])
pf.observe(updateLuminance, names=["luminance"])

print("Phidget widget is ready.")

### Step 1b.

Next we set up the gauges and some buttons, using plotly and widgets. 

In [ ]:
from IPython.display import display, Javascript
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from datetime import datetime
from time import sleep
import threading

In [ ]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 1000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

## The moisture trigger setpoint

moist_slider = widgets.FloatSlider(
    value=.3,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Moisture setpoint:',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='400px', height='80px'),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# the watering button
water_button = widgets.Button(
    description='Click to water',
    tooltip='Water for two seconds',
    disabled=False,
    button_style='success',
)

# the auto-watering Start/Stop button
auto_button = widgets.Button(
    description='Start Auto Watering',
    tooltip='Start or stop the auto-watering',
    disabled=False,
    button_style='success',
)

record_text = widgets.Textarea(
    value='Record of the watering',
    description='Last watered:',
    disabled=False,
    layout=widgets.Layout(width='40%', height='80px')
)

print('Gauges and buttons ready')

### Step 1c. Connecting the buttons

We connect the buttons in the user interface to functions that access the Phidget sensors.

In [ ]:
## The button/slider user interface

## We connect the water button to the Phidget widget named pf
## We will also record when the watering happened
def water_plant(button):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    record_text.value = now + ', Moisture = ' + str(pf.moisture) + '\n' + record_text.value
    pf.send("pumpOn")
    button.button_style = 'info'
    button.description='Watering...'
    sleep(2)
    pf.send("pumpOff")
    button.button_style = 'success'
    button.button_style = 'success'
    button.description='Click to water'

water_button.on_click(water_plant)

# conditionally water the plant, if the moisture is too low. We also record the event in the text box
def water_if_dry():
    if (gauges.data[2]['value'] < moist_slider.value):
        water_plant(water_button)

 # a flag to monitor if the auto-watering is on or not. The "event" is thread-safe
is_watering = threading.Event()
is_watering.clear()
       
# The watering loop will try to water each day, for up to 100 days
def watering_loop():
    for i in range(100):
        if (is_watering.is_set()):
            water_if_dry()
            for j in range(60*60*24):  # check every second of a day, to see if we continue auto-watering
                if not is_watering.is_set():
                    return
                sleep(1) # pause a second, then continue to count off seconds in the day
        else:
            break
    
# Click the auto button to start and stop the automatic watering
def auto_button_do(widget):
    global is_watering
    if (widget.description=='Start Auto Watering'):
        widget.description='Stop Auto Watering'
        widget.button_style='danger'
        is_watering.set()
        thread = threading.Thread(target=watering_loop) 
        thread.start()
    else:
        widget.description='Start Auto Watering'
        widget.button_style='success'
        is_watering.clear()

auto_button.on_click(auto_button_do)

print('buttons ready')

***
## Step 2. Viewing the user interface

We can view the user interface with the display command. We assemble them using the HBox and VBox functions


In [ ]:
dashboard = widgets.VBox([pf,gauges,moist_slider,widgets.HBox([water_button,auto_button]),record_text])
display(dashboard)

***
## Step 3: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes six separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the DC power supply, attached to Port 1 on the VINT device
- The water pump, attached to the DC power supply
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT
- optionally, the combined temperature/humidity sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you whether you wish to connect to this USB device. Please answer "yes" to this security request. 


### Step 3a. Opening the USB connection

Steps connecting your USB device to your Phidget VINT device to your computer. 
1. The VINT must be plugged into your computer's USB port. 
1. Click on the 'Connect' button in the user interface above.
1. Follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

 ### 3b. Check.
 Clicking the "Connect" button should open a window that asks you to select the Phidget Hub.  The window looks like this:

<img src="images/Pconnect.png" width=300>
<div align="left">
    
Go ahead and slect the Hub, and click "Connect."

### 3c. Confirm the USB connection 

The status line below the 'Connect button' will tell you if the USB and devices are connected.

If all is working well, the gauges will update with the latest readings from the sensors.

If nothing happens, make sure you have the VINT hub connected to your computer's USB port. You should also have the sensors connected to the VINT hub. 

Click "Disconnect" and then "Connect" again. Try reading the data a second time. Sometimes the Phidgets need a few seconds to connect properly, so disconnecting and connecting again can help. 

***
## Step 4: Watering your plant

Now that everything is connected, we can view the user interface above. 

The gauges should be live, showing the values from the sensors.

Try the  "Click to Water" button. It should run the water pump for two seconds.

Then try the "Start Auto Watering" button. This will check every day to see how moist the soil is. If the moisture is below the setpoint, as set by the slider, the computer will run water pump for two seconds. 

While Auto Watering is active, the button turns Red, and changes to say "Stop Auto Watering." Click on it if you want to stop the auto-watering. 

## Emergent pump off

If you need to turn off the water pump, run the following cell. 

In [ ]:
pf.send("pumpOff")

***
## Step 5: Observing the results

You'll need to be patient to see the results, as the code only checks the watering once a day. 

You can adjust the moisture setpoint at any time, using the slider. If you want to test it, just stop and re-start the auto-watering using the Auto Watering button. At start-up, the code always tests to see if the moisture gauge is reading below the set point. If it is below, then the pump gives a squirt of water.

Try running this for several days. Does the system keep your plant watered?

***
## 6. Final Step. Closing down the sensors

It is **important** to close down all the sensors when you are done, as otherwise they will keep busy forever. Also, this notebook will block other notebooks from using the same sensors.

You can disconnect by clicking the "Disconnect" button above. Or, simply close this notebook. 

#### Troubleshooting and/or not working?
If your are unable to read the data from the sensors, you can try the following:

1. Click disconnect and reconnect botton in this notebook
2. Check that the senors are connected to the VINT HUB Phidgets

## Conclusion

We have demonstrated the process of running a utility that retrieves Phidget sensor data and publishes it on the web. This operation involves a combination of Javascript and Python to interact with both the front-end and back-end components within the Jupyter notebook environment.

If you plan to collect data over an extended period, it's essential to have the Jupyter "Hub" software installed on your local computer. Please note that if you run this software on the [Callysto hub](https://hub.callysto.ca), it may automatically disconnect after approximately a day of continuous data collection.

You might also like to go back to the data collection notebooks: 
- [Collecting Data from Plants with Phidgets with Google Sheets](./plants-data-google-anywidget.ipynb)
- [Collecting Data from Plants with Phidgets with Ethercalc](./plants-data-ethercalc-anywidget.ipynb)


***
For additional support or inquiries related to this notebook, feel free to reach out to M. Lamoureux or Mary Grant, who serve as Callysto ambassadors. They can provide further details and assistance.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)